In [85]:
import getpass
import os
import re
import torch
import pandas as pd

from tqdm.auto import tqdm
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler
from sklearn.model_selection import train_test_split

In [34]:
PATH_DATA = "data/implicit-hate-corpus/"
PATH_INPUT_POSTS = "implicit_hate_v1_SAP_posts.tsv"
PATH_HATE_LABEL = "implicit_hate_v1_stg1.tsv"
PATH_CATEGORY_LABEL = "implicit_hate_v1_stg2.tsv"

# Implicit hate speech detection

Get help from this [kaggle page](https://www.kaggle.com/code/yousefadham/working-hatebert)

# Load the data

In [49]:
input = pd.read_csv(PATH_DATA + PATH_INPUT_POSTS, sep = '\t')
hate_labels = pd.read_csv(PATH_DATA + PATH_HATE_LABEL, sep = '\t')
category_labels = pd.read_csv(PATH_DATA + PATH_CATEGORY_LABEL, sep = '\t')

num_labels = 3

# Load the Hate Bert model

We decide to use the Hate Bert model, a Bert model specially trained to detect hate. This model can be use from hugging face [plateforme](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html).

In [38]:
model = AutoModelForSequenceClassification.from_pretrained(
    "GroNLP/hateBERT",
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load the tokenizer

From hugging face plateforme, we can also load the tokenizer specially made for Hate Bert

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")

## DataLoader initialization

In [40]:
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
        
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## Prepare the data

First let's visualise the data

In [65]:
input

,ID,post
0,SAP_10011,Guys I have a question: when a gay couple gets...
1,SAP_10036,Heard this from a 12 year old: 'Whats the diff...
2,SAP_10045,Friend: how dark is your humor? Me: If you whi...
3,SAP_10051,What do you say to a black guy in a three piec...
4,SAP_10085,how do you call a black man you call his cell ...
...,...,...
543,SAP_9953,How do you get a black man out of a tree? Cut ...
544,SAP_9960,Helen Keller walked into a bar and then a chai...
545,SAP_9962,Why do sumo wrestlers shave their legs? So tha...
546,SAP_9966,What's the difference between a black slave an...


In [64]:
hate_labels

,ID,class
0,399886440588247041,implicit_hate
1,929901925100937216,not_hate
2,728678509497954304,not_hate
3,625688620444180481,not_hate
4,441089979322597376,not_hate
...,...,...
21475,SAP_17154,implicit_hate
21476,SAP_17179,implicit_hate
21477,SAP_17314,implicit_hate
21478,SAP_17343,implicit_hate


We can notice that they are more classification than the number of input posts. -> Why is this like this ?  <br>
To overcome this issue, we remove every label that don't correpsond to a post.

In [67]:
hate_labels = hate_labels["ID"][hate_labels["ID"].isin(input["ID"])]

In [ ]:
#Set some general constant that will be used in the entire code
RANDOM_SEED = 42
MAX_LENGTH = 128 # -> What is this ? 

In [68]:
#Prepare the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    input, hate_labels, test_size=0.2, random_state=RANDOM_SEED
)

#Create the dataset
train_dataset = HateSpeechDataset(
    texts=train_texts,
    labels=train_labels,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)

#Create the test dataset
test_dataset = HateSpeechDataset(
    texts=test_texts,
    labels=test_labels,
    tokenizer=tokenizer,
    max_length=MAX_LENGTH
)

## Train the model

We use the default training configuration from the kaggle page

In [81]:
optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device("cpu")

# Calculate class weights
class_weights = torch.tensor([len(train_dataset) / (7 * 15294),  # toxic
                              len(train_dataset) / (7 * 1595),   # severe_toxic
                              len(train_dataset) / (7 * 8449),   # obscene
                              len(train_dataset) / (7 * 478),    # threat
                              len(train_dataset) / (7 * 7877),   # insult
                              len(train_dataset) / (7 * 1405),   # identity_hate
                              len(train_dataset) / (7 * 143346)  # normal
                             ]).to(device)

# Use weighted BCEWithLogitsLoss
criterion = BCEWithLogitsLoss(weight=class_weights)


model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Do we want to use a scheduler ? 

In [87]:
num_epochs = 2
num_training_steps = num_epochs * len(train_dataset)
# feel free to experiment with different num_warmup_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)

In [83]:
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True, pin_memory=True)

In [88]:
progress_bar = tqdm(range(num_training_steps))

# put the model in train mode
model.train()

# iterate over epochs
for epoch in range(num_epochs):
    # iterate over batches in training set
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        '''
        **kwargs is a common idiom to allow an arbitrary number of arguments to functions
        The **kwargs will give you all keyword arguments as a dictionary
        https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters
        '''
        

        outputs = model(**batch)
        '''
        Note that Transformers models all have a default task-relevant loss function,
        so you don’t need to specify one unless you want to

        Get the loss form the outputs
        in this example, the outputs are instances of subclasses of ModelOutput
        https://huggingface.co/transformers/v4.3.0/main_classes/output.html
        Those are data structures containing all the information returned by
        the model, but that can also be used as tuples or dictionaries.

        The outputs object has a loss and logits attribute
        You can access each attribute as you would usually do,
        and if that attribute has not been returned by the model, you will get None.
        for instance, outputs.loss is the loss computed by the model
        '''

        ### BEGIN SOLUTION
        loss = outputs.loss
        ### END SOLUTION

        # do the backward pass
        ### BEGIN SOLUTION
        loss.backward()
        ### END SOLUTION

        # perform one step of the optimizer
        ### BEGIN SOLUTION
        optimizer.step()
        ### END SOLUTION

        # peform one step of the lr_scheduler, similar with the optimizer
        ### BEGIN SOLUTION
        lr_scheduler.step()
        ### END SOLUTION

        # zero the gradients, call zero_grad() on the optimizer
        ### BEGIN SOLUTION
        optimizer.zero_grad()
        ### END SOLUTION

        progress_bar.update(1)

  0%|          | 0/876 [00:00<?, ?it/s]

KeyError: 333

In [ ]:
import evaluate

# define the metric you want to use to evaluate your model
metric = evaluate.load("accuracy")
progress_bar = tqdm(range(len(eval_dataloader)))

# put the model in eval mode
model_bert_l4.eval()
# iterate over batches of evaluation dataset
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        '''
        pass the batches to the model and get the outputs
        (hint: look at the training loop)
        outputs = ...
        '''
        ### BEGIN SOLUTION
        outputs = model_bert_l4(**batch)
        ### END SOLUTION

    
    '''
    get the logits from the outputs,
    similar as you did for the loss in the training loop
    logits = ...
    '''
    ### BEGIN SOLUTION
    logits = outputs.logits
    ### END SOLUTION

    # use argmax to get the predicted class
    predictions = torch.argmax(logits, dim=-1)
    
    '''
    metric.add_batch() adds a batch of predictions and references
    Metric.add_batch() by passing it your model predictions, and the references
    the model predictions should be evaluated against
    '''
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar.update(1)
# calculate a metric by  calling metric.compute()
metric.compute()

